## <center> Projet Risk Premia </center>

- Importation des packages

In [117]:
import math
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import time
import scipy.stats 


- Chargement des données

In [92]:
crsp_data = pd.read_csv("/Users/eloi/Desktop/Risk Premia/Project/Size premium data 2000-2023.csv")
FF = pd.read_csv("/Users/eloi/Desktop/Risk Premia/Project/F-F_Research_Data_Factors.txt")


In [93]:
crsp_data

,PERMNO,date,PRC,RET,SHROUT,vwretd
0,10026,2000-01-31,19.1250,-0.067073,9017.0,-0.039622
1,10026,2000-02-29,18.4375,-0.035948,9017.0,0.031760
2,10026,2000-03-31,19.8750,0.077966,8980.0,0.053499
3,10026,2000-04-28,15.9375,-0.198113,8810.0,-0.059519
4,10026,2000-05-31,14.8750,-0.066667,8810.0,-0.038863
...,...,...,...,...,...,...
535631,92807,2023-08-31,3.0200,-0.075758,17147.0,-0.020605
535632,92807,2023-09-29,2.6500,-0.122517,17147.0,-0.047390
535633,92807,2023-10-31,2.5500,-0.037736,17147.0,-0.028867
535634,92807,2023-11-30,2.9700,0.176471,17147.0,0.092057


In [94]:
FF

,YearMonth,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31
...,...,...,...,...,...
1166,202309,-5.24,-2.51,1.52,0.43
1167,202310,-3.19,-3.87,0.19,0.47
1168,202311,8.84,-0.02,1.64,0.44
1169,202312,4.85,6.34,4.93,0.43


- On remplace toutes les valeurs -99, -88, -66, B et C par des NaN puis on supprime toutes les valeurs NaN. 
- On convertit toutes les valeurs de la colonne 'PRC' en valeurs absolues car certains prix ont des signes négatifs  
(en l'absence de prix de clôture, l'identifiant CRSP d'une action reporte suivant les cas le dernier prix bid ou dernier prix ask de l'action et ce, avec un signe négatif)
- On crée une colonne période qui correspond à la différence entre l'année en cours et année minimale de la base (soit des périodes allant de 0 à 23)
- On crée une colonne 'YearMonth' qui comprend le mois et l'année de la colonne 'date'

In [95]:
NADict = {-99 : np.nan, -88: np.nan, -77:np.nan, -66:np.nan, 'B':np.nan, 'C':np.nan}
crsp_data = crsp_data.replace({'RET' : NADict})
crsp_data['RET'] = crsp_data['RET'].astype(float)
crsp_data = crsp_data.dropna()
crsp_data["PRC"]=crsp_data["PRC"].abs()
crsp_data["date"] = pd.to_datetime(crsp_data["date"])

minYear = crsp_data["date"].dt.year.min() # prenons l'année minimale dans la colonne "date"
crsp_data["period"] = crsp_data["date"].dt.year - minYear 
# Création d'une colonne "YearMonth"
crsp_data["YearMonth"] = crsp_data["date"].dt.strftime("%Y%m").astype(int)


In [96]:
crsp_data

,PERMNO,date,PRC,RET,SHROUT,vwretd,period,YearMonth
0,10026,2000-01-31,19.1250,-0.067073,9017.0,-0.039622,0,200001
1,10026,2000-02-29,18.4375,-0.035948,9017.0,0.031760,0,200002
2,10026,2000-03-31,19.8750,0.077966,8980.0,0.053499,0,200003
3,10026,2000-04-28,15.9375,-0.198113,8810.0,-0.059519,0,200004
4,10026,2000-05-31,14.8750,-0.066667,8810.0,-0.038863,0,200005
...,...,...,...,...,...,...,...,...
535631,92807,2023-08-31,3.0200,-0.075758,17147.0,-0.020605,23,202308
535632,92807,2023-09-29,2.6500,-0.122517,17147.0,-0.047390,23,202309
535633,92807,2023-10-31,2.5500,-0.037736,17147.0,-0.028867,23,202310
535634,92807,2023-11-30,2.9700,0.176471,17147.0,0.092057,23,202311


- On divise toutes les valeurs des colonnes 'Mkt-RF', 'SMB', 'HML', 'RF' par 100 car les chiffres donnés sont des pourcentages

In [97]:
FF[['Mkt-RF','SMB','HML','RF']] = FF[['Mkt-RF','SMB','HML','RF']]/100 


In [98]:
FF

,YearMonth,Mkt-RF,SMB,HML,RF
0,192607,0.0296,-0.0256,-0.0243,0.0022
1,192608,0.0264,-0.0117,0.0382,0.0025
2,192609,0.0036,-0.0140,0.0013,0.0023
3,192610,-0.0324,-0.0009,0.0070,0.0032
4,192611,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...,...
1166,202309,-0.0524,-0.0251,0.0152,0.0043
1167,202310,-0.0319,-0.0387,0.0019,0.0047
1168,202311,0.0884,-0.0002,0.0164,0.0044
1169,202312,0.0485,0.0634,0.0493,0.0043


- On fusionne les tableaux crsp_data (les taux de rentabilité mensuels des 2000 actions sur 2000-2023) et FF (fichier mensuel des facteurs Fama-French) en un seul tableau data

In [99]:
data = pd.merge(crsp_data, FF, on='YearMonth', how='inner')


In [100]:
data[data['period'] == 6]

,PERMNO,date,PRC,RET,SHROUT,vwretd,period,YearMonth,Mkt-RF,SMB,HML,RF
110793,10026,2006-01-31,30.28,0.019357,18315.0,0.040072,6,200601,0.0304,0.0539,0.0108,0.0035
110794,10032,2006-01-31,28.31,0.244943,44048.0,0.040072,6,200601,0.0304,0.0539,0.0108,0.0035
110795,10044,2006-01-31,13.93,-0.144139,6320.0,0.040072,6,200601,0.0304,0.0539,0.0108,0.0035
110796,10065,2006-01-31,13.03,0.038247,84090.0,0.040072,6,200601,0.0304,0.0539,0.0108,0.0035
110797,10138,2006-01-31,76.43,0.061086,131678.0,0.040072,6,200601,0.0304,0.0539,0.0108,0.0035
...,...,...,...,...,...,...,...,...,...,...,...,...
132599,91855,2006-12-29,31.40,-0.015893,14197.0,0.010857,6,200612,0.0087,-0.0114,0.0273,0.0040
132600,92583,2006-12-29,11.25,-0.136882,3223.0,0.010857,6,200612,0.0087,-0.0114,0.0273,0.0040
132601,92655,2006-12-29,53.73,0.094743,1346996.0,0.010857,6,200612,0.0087,-0.0114,0.0273,0.0040
132602,92690,2006-12-29,15.44,0.000648,10240.0,0.010857,6,200612,0.0087,-0.0114,0.0273,0.0040


- Fonction qui effectue la régression linéaire des betas 

In [101]:
def Function_Compute_Betas(data, yvar, xvar):
    Ri = data[yvar]
    Rm = data[xvar]
    if isinstance(Rm, pd.Series):
        Rm = Rm.to_frame()
    Rm['intercept'] = 1
    output = sm.OLS(Ri, Rm).fit()
    return output.params


Dans la fonction Function_FP_Betas_Portfolios : 
- on prend ttes les données de la colonne période comprises entre bornes inf et sup, i.e. données entre période 1 (janvier 2000) et période 3 (décembre 2002)
- on groupe ces données par PERMNO
- on estime les betas sur les données sélectionnées : on a les betas des actions estimés sur la période janvier 2000 - décembre 2002
- on crée un nouveau dataframe qui contient numéro du PF pr chq PERMNO
- diviser les betas en 5 quantiles (1er = les plus petits betas) : on a environ 310 betas par quantile et non plus 400 car on a enlevé des valeurs manquantes
- on fusionne les 2 tableaux par le facteur commun PERMNO : on a donc un 2e tableau qui contient les n° de portefeuilles en fonction des betas

In [102]:
def Function_FP_Betas_Portfolios(FP_Bounds):
    FP_data = data[data['period'].between(FP_Bounds[0], FP_Bounds[1], 'both')]
    by_PERMNO = FP_data.groupby('PERMNO')
    FP_Betas = pd.DataFrame(by_PERMNO.apply(Function_Compute_Betas,'RET', ['vwretd']))
    
                                                           
    del FP_Betas['intercept']
    FP_Betas.columns = ['beta_form'] 
    
    FP_Betas_Portfolios = pd.DataFrame(pd.qcut(FP_Betas['beta_form'], q=5, labels=False))   
    FP_Betas_Portfolios.columns = ['formPF']
    FP_Betas_Portfolios = pd.merge(FP_Betas_Portfolios, FP_Betas, on='PERMNO')
    return FP_Betas_Portfolios


Dans la fonction Function_EP_Betas : 
- on reprend le tableau de la fonction précédente
- on prend ttes les données de la colonne 'period'comprises entre bornes inf (janvier 2003) et borne sup (décembre 2005)
- on assigne à chaque action son portefeuille correspondant de la période d'estimation
- on construit un tableau constitué des colonnes PERMNO, beta_form, formPF et period
- on groupe les données par PERMNO et on crée un nouveau DataFrame qui contient les betas estimés sur la période d'estimation (janvier 2003 - décembre 2005) 
- Réestimation des betas fait que les erreurs de mesure sont indépendantes des erreurs sur les coeffs utilisés pour effectuer le regroupement
- Permet d'obtenir estimateur sans biais => on réestime mais sur une autre période
- ces nouveaux betas nous servir à calculer la moyenne pondérée des 25 PF
- on fusionne les tableaux des betas estimés pdt période de formation + données de la colonne 'period' comprises entre janv 2003 et dec 2005
- on groupe par PERMNO et on réestime les betas sur cette nvelle période pour obtenir  des estimateurs sans biais

In [103]:
def Function_EP_Betas(EP_Bounds, FP_Betas_Portfolios):
    EP_data = data[data['period'].between(EP_Bounds[0], EP_Bounds[1], 'both')]
    # assign to each stock (PERMNO) its corresponding portfolio in estimation period
    
    EP_data = pd.merge(EP_data, FP_Betas_Portfolios, how='left', on='PERMNO')
    
    by_PERMNO = EP_data.groupby('PERMNO')
    EP_Betas = pd.DataFrame(by_PERMNO.apply(Function_Compute_Betas,'RET', ['vwretd']))
    del EP_Betas['intercept'] # we don't need the alpha
    EP_Betas.columns = ['beta_est'] # we rename the column (initially 'vwretd' for clarity)

    return EP_Betas


Dans la fonction Function_MarketCap : 
    
- on filtre d'abord les données comprises entre janv 2003 et décembre 2005 (on estime la market cap en se basant sur la période d'estimation)
- on sélectionne ensuite toutes les actions où lignes de 'YearMonth' sont égales à 200512
- on calcule les market caps pour les PRC et SHROUT de décembre 2005 (prix * nombre d'actions en circulation pour décembre 2005)

In [104]:
def Function_MarketCap(EP_Bounds):
    EP_data = data[data['period'].between(EP_Bounds[0], EP_Bounds[1], 'both')]
    
    last_month = max(EP_data['YearMonth'])
    EP_data_last_month = EP_data[EP_data['YearMonth'] == last_month]
    EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
    last_month_market_cap = EP_data_last_month[['PERMNO','Market_Cap']]
      
    return last_month_market_cap


Dans la fonction Function_Last_Month_Data : 
- on obtient un tableau qui contient : PERMNO, formPF, beta_form, beta_est, Market_Cap

In [105]:
def Function_Last_Month_Data(FP_Betas_Portfolios,EP_Betas,last_month_market_cap):
    
    Last_Month_Data = pd.merge(FP_Betas_Portfolios, EP_Betas, on='PERMNO')
    
    Last_Month_Data = pd.merge(Last_Month_Data, last_month_market_cap, on='PERMNO')

    return Last_Month_Data


Dans la fonction Function_Create_Quintile_Portfolios : 
- on divise les market caps en 5 quintiles (1er quintile = small caps)
- on crée une liste vide
- la ligne quintile_portfolio = portfolio[quintiles == i] sélectionne les lignes de la liste portfolio où les valeurs de quintiles sont égales à i, i.e. le quintile actuel
- on assigne ensuite un nouveau numéro de portefeuille

In [106]:
def Function_Create_Quintile_Portfolios(portfolio, initial_portfolio_number):
    quintiles = pd.qcut(portfolio['Market_Cap'], 5, labels=False)
    portfolios = []
    for i in range(5):
        quintile_portfolio = portfolio[quintiles == i]
        quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
        portfolios.append(quintile_portfolio)
    return portfolios

Dans la fonction Function_Size_Portfolios : 
    
- on a d'abord trié par betas et ensuite par market cap, c'est pour ça qu'on doit assigner des nouv numéros de PF ! 
- on applique un 1er filtre (les betas) puis un 2eme filtre (les market caps)

In [107]:
def Function_Size_Portfolios(Last_Month_Data):
    new_portfolios = []
    
    for i in range(5):
        portfolio = Last_Month_Data[Last_Month_Data['formPF'] == i]
        quintile_portfolios = Function_Create_Quintile_Portfolios(portfolio, i)
        new_portfolios.extend(quintile_portfolios)
    
    Betas_and_Size_Portfolios = pd.concat(new_portfolios, ignore_index=True)
    return Betas_and_Size_Portfolios

In [108]:
# test du code avec 
FP_Bounds = (0, 2) 
EP_Bounds = (3, 5)
T_Bounds = (6, 6)

In [109]:
FP_Betas_Portfolios = Function_FP_Betas_Portfolios(FP_Bounds)
EP_Betas = Function_EP_Betas(EP_Bounds, FP_Betas_Portfolios)
last_month_market_cap = Function_MarketCap(EP_Bounds)
Last_Month_Data = Function_Last_Month_Data(FP_Betas_Portfolios,EP_Betas,last_month_market_cap)
Betas_and_Size_Portfolios= Function_Size_Portfolios(Last_Month_Data)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker

In [110]:

print(FP_Betas_Portfolios)
print(EP_Betas)
print(last_month_market_cap)
print(Last_Month_Data)
print(Betas_and_Size_Portfolios)



        formPF  beta_form
PERMNO                   
10026        2   0.438164
10032        4   3.170427
10044        2   0.564179
10065        3   1.012794
10138        3   1.366832
...        ...        ...
91855        1   0.335159
92583        4   1.829877
92655        1   0.159756
92690        0  -0.190422
92807        2   0.631903

[1534 rows x 2 columns]
        beta_est
PERMNO          
10026   1.159280
10032   1.688153
10044   1.203509
10065   0.855444
10138   1.719001
...          ...
91855   1.115364
92583   1.601442
92655   0.357751
92690   0.546088
92807   0.186922

[1781 rows x 1 columns]
        PERMNO   Market_Cap
109017   10026    543542.09
109018   10032   1001651.52
109019   10044    102864.32
109020   10065   1055329.50
109021   10138   9484766.34
...        ...          ...
110788   91855    396857.90
110789   92583     17876.15
110790   92655  84821100.00
110791   92690    154118.70
110792   92807     96850.56

[1776 rows x 2 columns]
      PERMNO  formPF  beta_for

Dans la fonction Function_PF_Betas_Market_Cap : 
- on utilise le tableau Betas_and_Size_Portfolios de la fonction précédente
- on crée une colonne 'PF_Betas' qui comprend les betas moyens estimés par n° de portefeuille
- on crée une colonne 'log_PF_market_cap qui comprend les log market capitalisations par n° de portefeuille


In [111]:
def Function_PF_Betas_Market_Cap(Betas_and_Size_Portfolios):
    Betas_and_Size_Portfolios['PF_Betas'] = Betas_and_Size_Portfolios.groupby('newPF')['beta_est'].transform('mean')
    Betas_and_Size_Portfolios['log_PF_market_cap'] = Betas_and_Size_Portfolios.groupby('newPF')['Market_Cap'].transform(lambda x: np.log(x).sum())    

    return Betas_and_Size_Portfolios


Dans la fonction Function_Period_Data : 
- on crée un tableau data_TR qui comprend les données pour la période souhaitée (précisée par n)
- on fusionne ce tableau avec le tableau en output de la fonction précédente, sur la base des PERMNO

In [64]:
def Function_Period_Data(n):
    data_TR = crsp_data[crsp_data["period"] == n]
    data_TR = pd.merge(data_TR, Betas_and_Size_Portfolios, on='PERMNO')
    return data_TR


Dans la fonction Function_NewPF_Annual_Return : 
- on sélectionne la période de test : de janvier à décembre de l'année n 
- on obtient un tableau avec les colonnes 'PERMNO', 'form_PF', 'beta_form', 'beta_est', 'Market_Cap', 'new_PF', 'PF_Betas', 'Log_PF_Market_Cap' pour l'année n 
- on crée une nouvelle colonne 'PF_monthly_return' qui donne le return mensuel moyen de chaque portefeuille et par mois (sur l'année 2006)
- on garde uniquement les colonnes : 'YearMonth', 'newPF' et 'PF_monthly_return' et on supprime 'PERMNO' car on n'en a plus besoin à ce moment la
- on calcule le return sur l'année par n° de PF (25 returns annuels uniques)


In [65]:
def Function_NewPF_Annual_Return(T_Bounds,Betas_and_Size_Portfolios,data):
    
    ret = pd.merge(Betas_and_Size_Portfolios, data, on='PERMNO')
    ret = ret[ret['period'].between(T_Bounds[0], T_Bounds[1], 'both')]
    ret['PF_monthly_return'] = ret.groupby(['newPF', 'YearMonth'])['RET'].transform('mean')   
    ret.drop(columns=['PERMNO'], inplace= True)
    ret_pf = ret[['YearMonth','newPF','PF_monthly_return']]
    ret['annuel_return'] = ret_pf.groupby('newPF')['PF_monthly_return'].transform('mean')
    ret = ret.drop_duplicates()
    ret
    return ret


In [123]:
FP_Bounds = (0, 2) # période de formation : janvier 2000 à décembre 2002 (n-6 à n-4)
EP_Bounds = (3, 5) # période d'estimation : janvier 2003 à décembre 2005 (n-3 à n-1)
TP_Bounds = (6, 6) # période de test : janvier 2006 à décembre 2006 (n)

FP = 3 # nombre de périodes de formation
EP = 3 # nombre de périodes d'estimation
TP = 1 # nombre de périodes de test

#À partir de l'année 2006 on esffectue le calcul du taux de rentabilité moyen des 25 portefeuilles 
#sur les 12 mois de l'année ce qui nous amène .


nTests = (data['period'].max()+1) - (FP + EP + TP) + 1 # on a 24 périodes - (3 + 3) + 1 = 18 tests à effectuer
print(str(nTests) + str(' tests à effectuer'))

18 tests à effectuer


In [67]:
def Function_Reg_Items(data_TR, month):
    # Initialiser les vecteurs pour les variables de régression. Ces vecteurs serviront à construire la matrice des variables explicatives.
    const_vec = np.ones((25,1))  # Vecteur de constantes pour inclure le terme intercept dans la régression.
    taux_renta_vec = np.zeros((25,1))  # Vecteur pour stocker les rentabilités moyennes des portefeuilles.
    betas_pf_vec = np.zeros((25,1))  # Vecteur pour stocker les bêtas moyens des portefeuilles.
    log_market_cap_vec = np.zeros((25,1))  # Vecteur pour stocker les logarithmes des capitalisations boursières moyennes.

    
    # Parcourir chaque portefeuille pour calculer les moyennes mensuelles nécessaires pour la régression.
    for i in data_TR["newPF"].unique():
        
        # Calculer la rentabilité moyenne pour le portefeuille 'i' pendant le mois spécifié.
        taux_renta_vec[i] = data_TR.loc[(data_TR["newPF"] == i) & (data_TR["date"].dt.month == month), "RET"].mean()
        # Calculer le bêta moyen pour le portefeuille 'i' pendant le mois spécifié.
        betas_pf_vec[i] = data_TR.loc[(data_TR["newPF"] == i) & (data_TR["date"].dt.month == month), "beta_est"].mean()
        # Calculer le logarithme moyen de la capitalisation boursière pour le portefeuille 'i' pendant le mois spécifié.
        log_market_cap_vec[i] = np.log(data_TR.loc[(data_TR["newPF"] == i) & (data_TR["date"].dt.month == month), "Market_Cap"]).mean()

    # Assembler les vecteurs préparés dans une matrice pour former les variables explicatives de la régression.
    matrix = pd.concat([pd.DataFrame(const_vec), pd.DataFrame(betas_pf_vec), pd.DataFrame(log_market_cap_vec)], axis=1)
    matrix.columns = ['Constante', 'Betas_PF', 'Log_MC']

    # Créer un DataFrame pour les rentabilités, qui servira de variable dépendante dans la régression linéaire.
    taux_renta = pd.DataFrame(taux_renta_vec)
    taux_renta.columns = ["TR_PF"] 

    # Effectuer la régression linéaire avec la méthode des moindres carrés ordinaires (OLS).
    outcome = sm.OLS(taux_renta, matrix).fit()

    # Extraire les coefficients de la régression et leurs écarts-types pour analyser l'impact des variables.
    coeffs = outcome.params
    std_dev = outcome.bse

    # Préparer un DataFrame pour retourner les résultats, incluant les coefficients et leurs écarts-types.
    results = pd.DataFrame({
        "constante": [coeffs[0]],
        "écart type de la constante": [std_dev[0]],
        "Beta": [coeffs[1]],
        "écart type du Beta": [std_dev[1]],
        "log market cap": [coeffs[2]],
        "écart type de la log market cap": [std_dev[2]]})

    return results

In [68]:
FP = 3  # Durée de la période de formation en années
EP = 3  # Durée de la période d'estimation en années
TP = 1  # Durée de la période de test en années

# Calcul du nombre total de périodes de test basé sur les données disponibles
nTests = (data['period'].max() + 1) - (FP + EP + TP) + 1

# Initialisation de listes pour stocker les résultats des coefficients de régression et les retours des portefeuilles
gammas_list = []  
all_periods_data = []  

# Boucle principale pour exécuter les analyses pour chaque période de test définie
for iter in range(0, nTests):
    print("\n ****** Iteration: " + str(iter) + "/" + str(nTests) + " ******\n")
    
    # Détermination des limites de chaque période en fonction de l'itération actuelle
    FP_Bounds = (iter, iter + FP - 1)
    EP_Bounds = (FP_Bounds[1] + 1, FP_Bounds[1] + EP)
    TP_Bounds = (EP_Bounds[1] + 1, EP_Bounds[1] + TP)
    
    print("formation period: " + str(FP_Bounds))
    print("estimation period: " + str(EP_Bounds))
    
    # Calcul des bêtas des portefeuilles pendant la période de formation
    FP_Betas_Portfolios = Function_FP_Betas_Portfolios(FP_Bounds)
    # Calcul des bêtas des portefeuilles pendant la période d'estimation
    EP_Betas = Function_EP_Betas(EP_Bounds, FP_Betas_Portfolios)
    # Calcul de la capitalisation boursière au dernier mois de la période d'estimation
    last_month_market_cap = Function_MarketCap(EP_Bounds)
    # Préparation des données finales du dernier mois pour les portefeuilles
    Last_Month_Data = Function_Last_Month_Data(FP_Betas_Portfolios, EP_Betas, last_month_market_cap)
    
    # Formation des portefeuilles basés sur les bêtas et la taille (capitalisation)
    Betas_and_Size_Portfolios = Function_Size_Portfolios(Last_Month_Data)
    # Calcul des bêtas finaux et des capitalisations des portefeuilles
    Betas_and_Size_Portfolios = Function_PF_Betas_Market_Cap(Betas_and_Size_Portfolios)
    
    # Préparation des données pour effectuer les régressions pour chaque mois de la période de test
    data_TR = Function_Period_Data(iter)
    
    # Exécution des régressions pour chacuns des mois de l'année du test en cours et stockage 
    #des résultats obtenus dans une liste de dataframes
    for month in range(1, 13):
        reg_params = Function_Reg_Items(data_TR, month)
        reg_params['Year'] = 2006 + iter
        reg_params['Month'] = month
        gammas_list.append(reg_params)
    
    # Calcul des rendements annuels des portefeuilles pour la période de test
    ret = Function_NewPF_Annual_Return(TP_Bounds, Betas_and_Size_Portfolios, data)
    all_periods_data.append(ret)  

# Concaténation des résultats des régressions et des rendements dans des DataFrames finaux pour l'analyse
final_ret = pd.concat(all_periods_data, ignore_index=True)
final_gammas = pd.concat(gammas_list, ignore_index=True)

print("\nfinished\n")  


 ****** Iteration: 0/17 ******

formation period: (0, 2)
estimation period: (3, 5)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


 ****** Iteration: 1/17 ******

formation period: (1, 3)
estimation period: (4, 6)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


 ****** Iteration: 2/17 ******

formation period: (2, 4)
estimation period: (5, 7)


...


 ****** Iteration: 16/17 ******

formation period: (16, 18)
estimation period: (19, 21)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


 ****** Iteration: 17/17 ******

formation period: (17, 19)
estimation period: (20, 22)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


finished



Visualisation des DataFrame obtenus :

In [69]:
final_ret

,formPF,beta_form,beta_est,Market_Cap,newPF,PF_Betas,log_PF_market_cap,date,PRC,RET,SHROUT,vwretd,period,YearMonth,Mkt-RF,SMB,HML,RF,PF_monthly_return,annuel_return
0,0,-0.015906,0.197209,68010.31,0,0.572264,674.574886,2006-01-31,42.785,-0.055727,1501.0,0.040072,6,200601,0.0304,0.0539,0.0108,0.0035,0.041704,0.009912
1,0,-0.015906,0.197209,68010.31,0,0.572264,674.574886,2006-02-28,40.000,-0.065093,1501.0,-0.001639,6,200602,-0.0030,-0.0038,-0.0034,0.0034,0.010790,0.009912
2,0,-0.015906,0.197209,68010.31,0,0.572264,674.574886,2006-03-31,40.500,0.012500,1501.0,0.019065,6,200603,0.0146,0.0344,0.0060,0.0037,0.030198,0.009912
3,0,-0.015906,0.197209,68010.31,0,0.572264,674.574886,2006-04-28,40.875,0.009259,1501.0,0.013000,6,200604,0.0073,-0.0142,0.0234,0.0036,-0.003237,0.009912
4,0,-0.015906,0.197209,68010.31,0,0.572264,674.574886,2006-05-31,42.000,0.027523,1501.0,-0.031032,6,200605,-0.0357,-0.0296,0.0241,0.0043,-0.006695,0.009912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405283,4,1.643513,1.233835,11775544.50,24,1.472525,1150.991785,2023-08-31,87.750,0.008621,138727.0,-0.020605,23,202308,-0.0239,-0.0316,-0.0106,0.0045,-0.020430,0.024082
405284,4,1.643513,1.233835,11775544.50,24,1.472525,1150.991785,2023-09-29,83.040,-0.053675,138727.0,-0.047390,23,202309,-0.0524,-0.0251,0.0152,0.0043,-0.042879,0.024082
405285,4,1.643513,1.233835,11775544.50,24,1.472525,1150.991785,2023-10-31,76.550,-0.075987,138727.0,-0.028867,23,202310,-0.0319,-0.0387,0.0019,0.0047,-0.069454,0.024082
405286,4,1.643513,1.233835,11775544.50,24,1.472525,1150.991785,2023-11-30,88.860,0.160810,135987.0,0.092057,23,202311,0.0884,-0.0002,0.0164,0.0044,0.092314,0.024082


In [70]:
final_gammas

,constante,écart type de la constante,Beta,écart type du Beta,log market cap,écart type de la log market cap,Year,Month
0,0.256350,0.081632,0.090310,0.021073,-0.025552,0.005677,2006,1
1,0.196025,0.181432,0.324133,0.046940,-0.037816,0.012611,2006,2
2,-0.312500,0.065161,-0.014243,0.016912,0.027022,0.004529,2006,3
3,-0.132046,0.060950,-0.101046,0.015767,0.017143,0.004239,2006,4
4,-0.107576,0.046570,-0.083326,0.011921,0.014312,0.003247,2006,5
...,...,...,...,...,...,...,...,...
211,-0.067098,0.019754,-0.006837,0.010420,0.004391,0.001148,2023,8
212,-0.025439,0.028064,0.115080,0.014804,-0.003146,0.001631,2023,9
213,-0.049077,0.017054,0.004731,0.008996,0.003744,0.000991,2023,10
214,-0.024976,0.042719,0.038912,0.022534,0.000902,0.002483,2023,11


In [71]:
#Question 4.1 (calcul des t-stats) basées sur final_gammas
t_stat_Constante = final_gammas["constante"].mean() * math.sqrt(len(final_gammas)) / final_gammas["constante"].std()
t_stat_Beta_PF = final_gammas["Beta"].mean() * math.sqrt(len(final_gammas)) / final_gammas["Beta"].std()
t_stat_Log_MC = final_gammas["log market cap"].mean() * math.sqrt(len(final_gammas)) / final_gammas["log market cap"].std()


In [72]:
t_stats_results = pd.DataFrame({"Constante" : t_stat_Constante,
                               "Beta" : t_stat_Beta_PF,
                               "Log MC" : t_stat_Log_MC}, index = ["T-stats"])
t_stats_results


,Constante,Beta,Log MC
T-stats,-0.853492,1.130983,1.66805


In [73]:
# Question 4.2 : on enlève le mois de janvier et on refait les même tests

FP = 3
EP = 3
TP = 1

nTests = (data['period'].max()+1) - (FP + EP + TP) + 1

gammas_list = []
all_periods_data = []  

for iter in range(0, nTests): 
    
    print("\n ****** Iteration: " + str(iter) + "/" + str(nTests) + " ******\n")
    FP_Bounds = (iter, iter + FP - 1)
    EP_Bounds = (FP_Bounds[1] + 1, FP_Bounds[1] + EP)
    TP_Bounds = (EP_Bounds[1] + 1, EP_Bounds[1] + TP)
    
    # 3.1 1) Années n − 6 à n − 4 (36 mois) : estimation du β de chacune des 2000 actions à l'aide du
            #modèle de marché et formation de 5 portefeuilles correspondant aux 5 quintiles de β des
            #actions
    
    print("formation period: " + str(FP_Bounds))
    FP_Betas_Portfolios = Function_FP_Betas_Portfolios(FP_Bounds)
    
    # 3.1 2) Années n − 3 à n − 1 (36 mois) : ré-estimation du β de chacune des actions
    
    print("estimation period: " + str(EP_Bounds))
    EP_Betas = Function_EP_Betas(EP_Bounds, FP_Betas_Portfolios)
    
    #3.1 3)
    #Année n − 1 : au mois de décembre, récupération de la capitalisation boursière de chacune
    #des actions

    last_month_market_cap = Function_MarketCap(EP_Bounds)
    
    
    Last_Month_Data = Function_Last_Month_Data(FP_Betas_Portfolios, EP_Betas, last_month_market_cap)
    Betas_and_Size_Portfolios = Function_Size_Portfolios(Last_Month_Data)
    Betas_and_Size_Portfolios = Function_PF_Betas_Market_Cap(Betas_and_Size_Portfolios)
    
    
    data_TR = Function_Period_Data(iter)
    
    for month in range(2, 13):      #On itère sur tout les mois en excluant le mois de janvier (i=1)
        reg_params = Function_Reg_Items(data_TR, month)
        reg_params['Year']= 2006 + iter
        reg_params['Month'] = month 
        gammas_list.append(reg_params)  

    ret = Function_NewPF_Annual_Return(TP_Bounds, Betas_and_Size_Portfolios, data)
    all_periods_data.append(ret)  

final_ret = pd.concat(all_periods_data, ignore_index=True)
final_gammas = pd.concat(gammas_list, ignore_index=True)
print("\nfinished\n")



 ****** Iteration: 0/17 ******

formation period: (0, 2)
estimation period: (3, 5)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


 ****** Iteration: 1/17 ******

formation period: (1, 3)
estimation period: (4, 6)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


 ****** Iteration: 2/17 ******

formation period: (2, 4)
estimation period: (5, 7)


...


 ****** Iteration: 16/17 ******

formation period: (16, 18)
estimation period: (19, 21)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


 ****** Iteration: 17/17 ******

formation period: (17, 19)
estimation period: (20, 22)


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1150550422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EP_data_last_month['Market_Cap'] = EP_data_last_month['PRC'] * EP_data_last_month['SHROUT']
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1881366706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintile_portfolio['newPF'] = initial_portfolio_number * 5 + i  # Assigner un nouveau numéro de portefeuille
/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipyker


finished



In [114]:
#Question 4.2

t_stat_Constante = final_gammas["constante"].mean() * math.sqrt(len(final_gammas)) / final_gammas["constante"].std()
t_stat_Beta_PF = final_gammas["Beta"].mean() * math.sqrt(len(final_gammas)) / final_gammas["Beta"].std()
t_stat_Log_MC = final_gammas["log market cap"].mean() * math.sqrt(len(final_gammas)) / final_gammas["log market cap"].std()


In [115]:
t_stats_results = pd.DataFrame({"Constante" : [t_stat_Constante],
                               "Beta_PF" : [t_stat_Beta_PF],
                               "Log MC" : [t_stat_Log_MC]}, index = ["T-stats"])
t_stats_results

,Constante,Beta_PF,Log MC
T-stats,-3.021897,1.325136,3.437084


Pour la question 4.3, on a calculé le rendement mensuel moyen (de 2006 à 2023) pour chacun des 25 portefeuilles constitués, en groupant les données selon l'identifiant de portefeuille (newPF) et en prenant la moyenne des rendements mensuels. Le résultat est stocké dans un DataFrame

In [113]:
# Question 4.3

PF_mensuel_moyen = pd.DataFrame({"Average return" : final_ret.groupby("newPF")["PF_monthly_return"].mean()})
PF_mensuel_moyen


,Average return
newPF,
0,0.013537
1,0.006569
2,0.006499
3,0.009353
4,0.008307
5,0.011935
6,0.009834
7,0.009621
8,0.008939


Pour la question 4.4 on à commencé par convertir la colonne YearMonth en format date pour faciliter le traitement ultérieur. Ensuite, on à calculé la rentabilité moyenne mensuelle pour chacun des 25 portefeuilles, regroupés par identifiant de portefeuille (newPF) et par mois de l'année. 

Les résultats sont organisés dans un DataFrame

In [126]:
# Question 4.4

# Calculer la rentabilité moyenne pour chaque portefeuille pour chaque mois sur toutes les périodes (question 4.4)
final_ret['YearMonth'] = pd.to_datetime(final_ret['YearMonth'], format="%Y%m")  # Ajuste le format de la date pour pouvoir 
monthly_average_returns = final_ret.groupby(['newPF', final_ret['YearMonth'].dt.month])['PF_monthly_return'].mean()

monthly_average_returns_df

newPF,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
YearMonth,,,,,,,,,,,,,,,,,,,,,
1,0.057595,0.022409,0.018027,0.020873,0.012525,0.054569,0.006643,0.005320,0.009826,0.001953,...,0.050019,0.004758,0.007503,0.009235,0.006936,0.082499,0.018026,-0.002276,0.008515,0.012165
2,0.014335,0.000185,-0.005027,0.003883,-0.000943,0.009846,-0.001853,-0.000451,0.000895,0.003099,...,0.015074,0.002759,0.008464,0.012718,0.004463,0.017082,0.011185,0.013472,0.013111,0.018184
3,0.002986,-0.000774,0.002532,0.008201,0.020036,0.003967,0.009188,0.016380,0.006536,0.015917,...,0.005913,0.019180,0.016975,0.011482,0.011701,0.020028,0.014168,0.004930,0.005746,0.006491
4,0.036828,0.017161,0.014851,0.017636,0.023412,0.023530,0.027198,0.022133,0.022264,0.023691,...,0.038006,0.033338,0.025313,0.033663,0.033080,0.028539,0.038698,0.032186,0.034383,0.037011
5,0.006261,0.007529,0.006583,0.005363,0.001631,0.004281,0.002052,-0.002731,0.003013,0.002846,...,0.003715,-0.002787,-0.000826,0.000246,-0.002031,0.000923,0.001041,-0.000788,-0.004115,-0.005537
6,0.002993,0.001798,0.005010,0.009113,-0.000623,0.008361,0.010232,0.004711,0.003166,-0.002030,...,0.005351,0.006157,0.003617,-0.000799,-0.005818,0.004791,0.017991,0.003564,0.000751,-0.005549
7,0.022097,0.011191,0.011785,0.016577,0.017084,0.024534,0.014605,0.021222,0.016020,0.020580,...,0.015583,0.021904,0.028152,0.020322,0.025886,0.016241,0.022151,0.019488,0.022909,0.022725
8,0.002013,-0.003426,0.002723,0.000549,-0.002397,0.000266,0.002615,0.000999,-0.000529,-0.002083,...,-0.004817,-0.003234,-0.004822,-0.001707,-0.001949,0.007047,-0.006521,-0.006516,-0.006075,-0.002003
9,-0.004146,0.000103,-0.008007,-0.009655,-0.011035,-0.004954,-0.001281,-0.002868,-0.007344,-0.004723,...,-0.003950,-0.005064,-0.009306,-0.010053,-0.009888,0.000537,-0.010600,-0.009172,-0.006655,-0.010203


Pour la question 4.5, on va procéder de deux manières:
- la première méthode utilise le dataframe obtenu à la question 4.4 (monthly_average_returns_df qui contient la     rentabilité moyenne pour chaque portefeuille pour chaque mois sur toutes les périodes)
- La seconde méthode se base sur le dataframe final_ret qui est celui que l'on obtient après la boucle itérant     sur toutes les périodes de test (2006 à 2023)

In [134]:
#Première méthode
from scipy.stats import ttest_ind

# Extraction des données nécessaires
january_first_quintile = monthly_average_returns_df.loc[1, 0:4].mean()  # Moyenne du premier quintile de janvier
january_all_other_quintiles = monthly_average_returns_df.loc[1, 5:].mean()  # Moyenne de tous les autres quintiles de janvier
january_last_quintile = monthly_average_returns_df.loc[1, 20:].mean()  # Moyenne du dernier quintile de janvier
first_quintiles_other_months = monthly_average_returns_df.iloc[1:, 0:4].mean()  # Moyenne du premier quintile pour tous les autres mois

# Test 1: Premier quintile de janvier vs tous les autres quintiles de janvier
t_stat_all, p_value_all = stats.ttest_ind(monthly_average_returns_df.loc[1, 0:4], monthly_average_returns_df.loc[1, 5:])

# Test 2: Premier quintile de janvier vs dernier quintile de janvier
t_stat_last, p_value_last = stats.ttest_ind(monthly_average_returns_df.loc[1, 0:4], monthly_average_returns_df.loc[1, 20:])

print("Test 1 - Premier quintile de janvier vs tous les autres quintiles de janvier:")
print("Statistique t:", t_stat_all, "P-value:", p_value_all)

print("Test 2 - Premier quintile de janvier vs dernier quintile de janvier:")
print("Statistique t:", t_stat_last, "P-value:", p_value_last)

Test 1 - Premier quintile de janvier vs tous les autres quintiles de janvier:
Statistique t: 0.8777438710129942 P-value: 0.38915982134702065
Test 2 - Premier quintile de janvier vs dernier quintile de janvier:
Statistique t: 0.14664985138521525 P-value: 0.8870366224177353


In [133]:
# Convertir 'date' en datetime 
final_ret['date'] = pd.to_datetime(final_ret['date'])

# Filtrer pour obtenir les données de janvier de toutes les années
january_data = final_ret[final_ret['date'].dt.month == 1]

# Création des quintiles de capitalisation pour les données de janvier
january_data['quantile'] = pd.qcut(january_data['Market_Cap'], 5, labels=False)

# Rendements du premier quintile (petite capitalisation)
january_first_quintile_returns = january_data[january_data['quantile'] == 0]['RET']

# Rendements du dernier quintile (grande capitalisation)
january_last_quintile_returns = january_data[january_data['quantile'] == 4]['RET']

# Test 1: Premier quintile de janvier vs tous les autres quintiles de janvier
t_stat_1, p_value_1 = ttest_ind(january_first_quintile_returns, january_data[january_data['quantile'] != 0]['RET'])

# Test 2: Premier quintile de janvier vs dernier quintile de janvier
t_stat_2, p_value_2 = ttest_ind(january_first_quintile_returns, january_last_quintile_returns)

print("Test 1 - Premier quintile de janvier vs tous les autres quintiles de janvier:")
print(f"Statistique t: {t_stat_1}, P-value: {p_value_1}")

print("Test 2 - Premier quintile de janvier vs dernier quintile de janvier:")
print(f"Statistique t: {t_stat_2}, P-value: {p_value_2}")

Test 1 - Premier quintile de janvier vs tous les autres quintiles de janvier:
Statistique t: 25.196244367233767, P-value: 8.352756565696957e-139
Test 2 - Premier quintile de janvier vs dernier quintile de janvier:
Statistique t: 16.63978341885526, P-value: 1.4606235448782446e-61


/var/folders/cj/s0zj0cz9019fmj09lc45yn040000gn/T/ipykernel_67697/1785976685.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  january_data['quantile'] = pd.qcut(january_data['Market_Cap'], 5, labels=False)
